## Номер документа

Номер документа часто указывается после символа No или N и может состоять из цифр, букв и спецсимволов. Примеры номеров:

* 67-ФЗ,

* 324-01-ЗМО,

* 824н.

Номер документа необходимо извлечь из текста «как есть», не подвергая дополнительной обработке.
Поле number должно содержать извлеченную строку. Чтобы скомпенсировать влияние ошибок распознавания символов в исходных текстах, при сравнении предсказанного и настоящего номера проверяющей системой регистр букв не учитывается.


Для каждого типа документа свой извлекатор:

* указ - в конце документа на отдельной строке № 807, может быть и в начале № 256-уг

* постановление, распоряжение, приказ - в начале после даты и места (или перед местом) в конце строки № 688-п, №_70-пп_, 63-пр,  № 25/2-г

* закон, федеральный закон - в конце документа № 89-ОЗ

Особые случаи:

* № _ 643-уГ

* 20 ноября 2014 года

№ 596-5-ЗКО

в постановлении в конце документа

* 02 ноября 2015 г. л 23/557-П в постановлении в начале (будем выбирать в качестве номера окончание строки)

In [1]:
import re
import json

In [303]:
LAW_EXPR = re.compile('[ _]\d\d\d\d[ _]+г[а-я\.]* ?.?[\n]+№[ _]*([\w\-/А-Яа-я]+)') # федеральный закон
SIMPLE_NUM = re.compile('№[_ ]*([\w\-/А-Яа-я]+)')

DECREE_EXPR = re.compile('\d\d\.\d\d\.\d\d\d\d[^\n]*[_ ]([\w\-/А-Яа-я]*\d+[\w\-/А-Яа-я]*)( -)?\n|'
                         '[_ ]\d\d\d\d[_ ]*[а-яА-Я\. \-]*№[_ ]([\w\-/А-Яа-я]*\d+[\w\-/А-Яа-я]*)( [Гг]\. [А-Яа-я]+)?( -)?\n|'
                         '\d\d\d\d[ _](г\.?|года)[ _](.[_ ])?([\w\-/А-Яа-я]*\d+[\w\-/А-Яа-я]*)\n') # постановление

ORDER_EXPR = re.compile('\n(от )?\d\d\.\d\d\.\d\d\d\d .? ?№ *([\w\-/А-Яа-я]+)\n|'
                        '\d\d\d\d (года|г.) № *([\w\-/А-Яа-я]+)\n|'
                        '\n№[_ ]*([\w\-/А-Яа-я]+)') # распоряжение

COMMAND_EXPR = re.compile('') # приказ

In [304]:
from typing import List

class NumberExtractor: 
    def predict(self, test: List[str], types: List[str]) -> List[str]:
        pred_numbers = []
        for doc, doc_type in zip(test, types):
            doc = doc
            pred_number = ''
            if doc_type == 'федеральный закон':
                matches = LAW_EXPR.findall(doc)
                if matches:
                    pred_number = matches[-1]
                else:
                    matches = SIMPLE_NUM.findall(doc)
                    if matches:
                        pred_number = matches[-1]
            elif doc_type == 'постановление':
                matches = DECREE_EXPR.findall(doc)
                if matches:
                    pred_number = matches[0][0] if matches[0][0] else matches[0][2]
                    pred_number = pred_number if pred_number else matches[0][7]
                else:
                    matches = SIMPLE_NUM.findall(doc)
                    if matches:
                        pred_number = matches[0]
            elif doc_type == 'распоряжение':
                matches = ORDER_EXPR.findall(doc)
                if matches:
                    pred_number = matches[0][1] if matches[0][1] else matches[0][3]
                    pred_number = pred_number if pred_number else matches[-1][4]
                    if matches[-1][4] and doc.endswith(matches[-1][4]):
                        pred_number = matches[-1][4]
            elif doc_type == 'приказ':
                pass
            pred_numbers.append(pred_number)
        return pred_numbers

In [305]:
from eval_module import quality
from type_extractor import TypeExtractor
from sklearn.model_selection import train_test_split

train = []
names =[]
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append((f.read(), doc_dict['label']))
# train_data, test, _, names_test = train_test_split(train, names)

test = train
test_data = [x[0] for x in test]
test_labels = [x[1] for x in test]
true_types = [x[1]['type'] for x in test]

def predict(pred_numbers):
    results = []
    for pred_number in pred_numbers:
        prediction = {"type": "",
                      "date": "",
                      "number": pred_number,
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

# te = TypeExtractor()
# te.train(train_data)

ne = NumberExtractor()
predicted = ne.predict(test_data, true_types)
predicted = predict(predicted)
quality(predicted, test_labels)

{'date_accuracy': 0.0,
 'number_accuracy': 0.7751316119,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [306]:
import numpy as np

true_types = np.array(true_types)
predicted = np.array(predicted)
test_labels = np.array(test_labels)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}: {quality(predicted[ids], test_labels[ids])['number_accuracy']}")

федеральный закон: 1.0
постановление: 0.948352989
приказ: 0.0
распоряжение: 0.9348837209
закон: 0.8341708543
указ: 0.0


In [307]:
doc_type = 'постановление'
ids = np.where(true_types == doc_type)

y_test = [(x['number'], x['type']) for x in test_labels[ids]]
y_pred = [x['number'] for x in predicted[ids]]
names = np.array(names)
test_names = names[ids]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0].lower() != pred.lower():
        wrong_docs.append((test_names[i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f.read())
        print("=====")

right value = ('41/112', 'приказ')
wrong value = 
РСа дЕ оаы е

УПРАВЛЕНИЕ ГОСУДАРСТВЕННОГО РЕГУЛИРОВАНИЯ ТАРИФОВ
БРЯНСКОЙ ОБЛАСТИ

 

НРИКАЗ

от 18 декабря 2015 года г. Брянск № 41/112 гвс

О тарифах на горячую воду

В соответствии с Федеральным законом от 07.12.2011 года № 416-ФЗ
"О водоснабжении и. водоотведении", постановлением Правительства
Российской Федерации от 13.05.2013 года № 406 "О государственном
регулировании тарифов в сфере водоснабжения и водоотведения",
Положением об управлении государственного регулирования тарифов
Брянской области, утвержденным указом Губернатора Брянской области от
28.01.2013 года № 45, приказываю:

1. Установить двухкомпонентный тариф на горячую воду в закрытой
системе - тёеплоснабжения - (горячего - водоснабжения) - поставляемую
потребителям МУП "Выгоничского ЖКХ" (котельная по адресу: П.
Выгоничи, ул. Новобрянская, д.1,3), согласно приложениям 1, 2.

2. Тарифы, установленные в пункте 1 настоящего приказа в части:

- приложение 1, действуют с 01 я